# Supplementary figure for the IBCM model on a log-normal background

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import seaborn as sns
import os, json
pj = os.path.join

In [ ]:
do_save_plots = True
# Resources
root_dir = pj("..", "..", "..")
data_folder = pj(root_dir, "results", "for_plots")
panels_folder = "panels/"
params_folder = pj(root_dir, "results", "common_params")

In [ ]:
# rcParams
with open(pj(params_folder, "olfaction_rcparams.json"), "r") as f:
    new_rcParams = json.load(f)
plt.rcParams.update(new_rcParams)

# color maps
with open(pj(params_folder, "back_colors.json"), "r") as f:
    all_back_colors = json.load(f)
back_color = all_back_colors["back_color"]
back_color_samples = all_back_colors["back_color_samples"]
back_palette = all_back_colors["back_palette"]

with open(pj(params_folder, "orn_colors.json"), "r") as f:
    orn_colors = json.load(f)
    
with open(pj(params_folder, "inhibitory_neuron_two_colors.json"), "r") as f:
    neuron_colors = np.asarray(json.load(f))
with open(pj(params_folder, "inhibitory_neuron_full_colors.json"), "r") as f:
    neuron_colors_full = np.asarray(json.load(f))

with open(pj(params_folder, "model_colors.json"), "r") as f:
    model_colors = json.load(f)
with open(pj(params_folder, "model_nice_names.json"), "r") as f:
    model_nice_names = json.load(f)

models = list(model_colors.keys())
print(models)

In [ ]:
n_neu = np.load(os.path.join(data_folder, 
                    "sample_lognormal_simulation.npz"))["cbarser"].shape[1]
print(n_neu)
n_components, n_orn = np.load(os.path.join(data_folder, 
                    "sample_lognormal_simulation.npz"))["back_vecs"].shape

In [ ]:
# Extra aesthetic parameters for this figure

# More legend rcParams: make everything smaller by 30 %
plt.rcParams["patch.linewidth"] = 0.75
legend_rc = {"labelspacing":0.5, "handlelength":2.0, "handleheight":0.7, 
             "handletextpad":0.8, "borderaxespad":0.5, "columnspacing":2.0}
for k in legend_rc:
    plt.rcParams["legend."+k] = 0.75 * legend_rc[k]

new_color = "r"
linestyles = ["-", "--", ":", (0, (5, 1, 2, 1)), "-."]
neuron_styles = linestyles + [(0, (1, 2, 1, 2))]

In [ ]:
def l2_norm(vecs, axis=-1):
    r""" Computes l2 norm of vectors stored along the last axis of vecs.
    Args:
        vecs can be either a single vector (1d) or an  arbitrary array of vectors,
            where the last dimension indexes elements of vectors.
        axis (int): which axis to sum along.

    Returns: array of distances of same shape as vecs
        except for the summation axis, removed.
    """
    return np.sqrt(np.sum(vecs**2, axis=axis))

## Panel A: OSN time series and concentration statistics

In [ ]:
# Load example concentration time series
ex = np.load(os.path.join(data_folder, "sample_lognormal_simulation.npz"))
skp_example = 1
tser_example = ex["tser"]  # Unskip this
save_skp = ex["skp"]
tser_example_unskp = np.arange(tser_example[0], tser_example[-1]+np.mean(np.diff(tser_example)), 
                               (tser_example[1]-tser_example[0])/save_skp)
ln10 = np.log(10.0)
                               
nuser_example = ex["nuser"]  # This has all time steps, for nicer histogram
nu0_avgs = ex["averages_nu0"]
conc_ser = np.exp(ln10 * (nuser_example + nu0_avgs))
n_components = conc_ser.shape[1]

# Time units for plotting, in ms
dt_u = 10.0  # ms

In [ ]:
# Plot time series and histogram. Choice 1: linear scales
fig = plt.figure()
gs = fig.add_gridspec(1, 3)
axes = [fig.add_subplot(gs[:2])]
axes.append(fig.add_subplot(gs[2], sharey=axes[0]))
# Plot time series
plot_skp = 20
t_window = slice(0, 16000, plot_skp)
tser_ex = tser_example_unskp[t_window]
conc_ser_ex = conc_ser[t_window]
ax = axes[0]
back_palette_6 = sns.light_palette("k", n_colors=6, reverse=True)
for i in range(0, n_components):
    ax.plot((tser_ex - tser_ex[0])*dt_u*skp_example/1000,
        conc_ser_ex[:, i],  color=back_palette_6[i], lw=0.5, label="Odor {}".format(i+1))
ax.set_xlabel("Time (s)")
ax.set_ylabel(r"Concentration $c_{\mu}(t)$", labelpad=0.0)
ax.legend(frameon=False, ncols=2, handlelength=1.0, columnspacing=1.0)
ax.set_ylim([0.0, 7.0])

# Plot a small histogram of all iid odors aggregated
counts, bins = np.histogram(conc_ser.flatten(), bins="sqrt", density=True)
ax = axes[1]
ax.barh(bins[:-1], counts, align="edge", height=bins[1:] - bins[:-1], color=back_color_samples)
ax.set(xlabel="Prob. density", xscale="log")
ax.tick_params(axis="x", which="major", pad=1.0)
ax.set_xticks([0.0001, 0.1])
[label.set_visible(False) for label in ax.get_yticklabels()]
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_lognormal_background_series.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

## Panel B: $M$ series

In [ ]:
back_vecs_example = ex["back_vecs"]
n_b = back_vecs_example.shape[0]
mbarser_example = ex["mbarser"]
cbarser_example = ex["cbarser"]
yser_example = ex["yser"]
yser_ibcm = ex["yser"]
wser_example = ex["wser"]
hgammaser_example = mbarser_example.dot(back_vecs_example.T)

# Analytical predictions
analytical_hs_hn = ex["hs_hn"]
analytical_h_saddle = ex["saddle_h"]
analytical_w = ex["analytical_w"]
specif_gammas = ex["specif_gammas"]
n_i_ibcm = mbarser_example.shape[1]

# Matrix of all dot products for each neuron with each background odor
analytical_hgammas = np.ones([n_i_ibcm, n_b]) * analytical_hs_hn[1]
analytical_hgammas[np.arange(n_i_ibcm), specif_gammas] = analytical_hs_hn[0]
print(specif_gammas)

In [ ]:
# Show three neurons
fig = plt.figure()
gs = fig.add_gridspec(3, 4)
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.25, 
                    plt.rcParams["figure.figsize"][1])
ax = fig.add_subplot(gs[:, :3])
axi = fig.add_subplot(gs[1, 3:])

#ax.axhline(0.0, ls="-", color=(0.8,)*3, lw=0.8)
t_axis = tser_example*dt_u/1000/60
legend_styles = [[0,]*6, [0,]*6, [0,]*6]
i_highlights = [2, 12, 20]  # Neurons to highlight
neuron_colors3 = neuron_colors_full[[8, 17, 23]]
clr_back = back_palette[-1]
plot_skp = 4
tslice = slice(0, t_axis.shape[0], plot_skp)

# plot all other neurons first, skip some points
for i in range(n_i_ibcm):
    if i in i_highlights: 
        continue
    else: 
        for j in range(n_b):
            ax.plot(t_axis[tslice], hgammaser_example[tslice, i, j], color=clr_back, 
                ls="-", alpha=1.0-0.1*j, lw=plt.rcParams["lines.linewidth"]-j*0.1)

# Now plot the highlighted neuron
for j in range(n_b):
    for i in range(len(i_highlights)):
        li, = ax.plot(t_axis[tslice], hgammaser_example[tslice, i_highlights[i], j], 
                      color=neuron_colors3[i], ls="-", alpha=1.0-0.1*j, 
                      lw=plt.rcParams["lines.linewidth"]-j*0.1)
        legend_styles[i][j] = li

# Annotate with analytical results
for j in range(2):
    ax.axhline(analytical_hs_hn[j], lw=1.0, ls="-.", color="k")
ax.annotate(r"Analytical $h_{\mathrm{ns}}$", xy=(t_axis[-5], analytical_hs_hn[1]-0.4), 
            ha="right", va="top", color="k", size=6)
ax.annotate(r"Analytical $h_\mathrm{sp}$", xy=(t_axis[-5], analytical_hs_hn[0]+0.4), 
            ha="right", va="bottom", color="k", size=6)
ax.set(xlabel="Time (min)", 
       ylabel=r"Alignments $\bar{h}_{i\gamma} = \mathbf{\bar{m}}_i \cdot \mathbf{s}_{\gamma}$")
ax.axhline(analytical_h_saddle, lw=1.0, ls=":", color="grey")
ax.annotate(r"$h_\mathrm{saddle}$", xy=(t_axis[-150], analytical_h_saddle+0.1), 
            ha="right", va="bottom", color="grey", size=6)

axi.tick_params(labelleft=True, labelbottom=False, labeltop=True)
for i in range(len(i_highlights)):
    for j in range(n_b):
        li = legend_styles[i][j]
        axi.plot([0.0+j, 0.5+j], [0.8*i, 0.8*i], color=li.get_color(), 
                 ls=li.get_linestyle(), alpha=li.get_alpha(), lw=li.get_linewidth())
for side in ["bottom", "left", "top", "right"]:
    axi.spines[side].set_visible(False)
axi.tick_params(axis="both", length=0, pad=3)
axi.set_xticks(np.arange(0.25, n_b+0.25, 2.0))
#axi.set_xticklabels([r"$\mathbf{\bar{m}}_i \cdot \mathbf{s}_a$", r"$\mathbf{\bar{m}} \cdot \mathbf{s}_b$"])
axi.set_xticklabels(list(range(0, n_b, 2)))
axi.set_xlabel(r"Odor $\gamma$", size=6, labelpad=3)
axi.xaxis.set_label_position('top') 
axi.set_yticks([0, 0.8, 1.6])
axi.invert_yaxis()
axi.set_yticklabels(["${}$".format(i) for i in i_highlights])
#axi.set_yticklabels(["Neuron 2", "Neuron 1"])
axi.set_ylabel("Neuron $i$", size=6, labelpad=1)

gs.tight_layout(fig, w_pad=-0.2)
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_lognormal_ibcm_m_series.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
# Alternate version with the legend inside the plot
# Show three neurons
fig, ax = plt.subplots()
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.05, 
                    plt.rcParams["figure.figsize"][1])

#ax.axhline(0.0, ls="-", color=(0.8,)*3, lw=0.8)
t_axis = tser_example*dt_u/1000/60
legend_styles = [[0,]*6, [0,]*6, [0,]*6]
i_highlights = [2, 12, 20]  # Neurons to highlight
neuron_colors3 = neuron_colors_full[[8, 17, 23]]
clr_back = back_palette[-1]
plot_skp = 4
tslice = slice(0, t_axis.shape[0], plot_skp)

# plot all other neurons first, skip some points
for i in range(n_i_ibcm):
    if i in i_highlights: 
        continue
    else: 
        for j in range(n_b):
            ax.plot(t_axis[tslice], hgammaser_example[tslice, i, j], color=clr_back, 
                ls="-", alpha=1.0-0.1*j, lw=plt.rcParams["lines.linewidth"]-j*0.1)

# Now plot the highlighted neuron
for j in range(n_b):
    for i in range(len(i_highlights)):
        li, = ax.plot(t_axis[tslice], hgammaser_example[tslice, i_highlights[i], j], 
                      color=neuron_colors3[i], ls="-", alpha=1.0-0.1*j, 
                      lw=plt.rcParams["lines.linewidth"]-j*0.1)
        legend_styles[i][j] = li

# Annotate with analytical results
for j in range(2):
    ax.axhline(analytical_hs_hn[j], lw=1.0, ls="-.", color="k")
ax.annotate(r"Analytical $h_{\mathrm{ns}}$", xy=(t_axis[1], analytical_hs_hn[1]-0.4), 
            ha="left", va="top", color="k", size=6)
ax.annotate(r"Analytical $h_\mathrm{sp}$", xy=(t_axis[1], analytical_hs_hn[0]+0.1), 
            ha="left", va="bottom", color="k", size=6)
ax.set(xlabel="Time (min)", 
       ylabel=r"Alignments $\bar{h}_{i\gamma} = \mathbf{\bar{m}}_i \cdot \mathbf{s}_{\gamma}$")
ax.axhline(analytical_h_saddle, lw=1.0, ls=":", color="grey")
ax.annotate(r"$h_\mathrm{saddle}$", xy=(t_axis[-150], analytical_h_saddle+0.1), 
            ha="right", va="bottom", color="grey", size=6)

# Need to do tight_layout before adding inset
fig.tight_layout()

# Custom legend: 2x3 table here, inset axes
axi = inset_axes(ax, width="50%", height="30%",
                   bbox_to_anchor=(.68, .33, .5, .5),  # left, bottom, width, height
                   bbox_transform=ax.transAxes, loc=3)

axi.tick_params(labelleft=True, labelbottom=False, labeltop=True)
for i in range(len(i_highlights)):
    for j in range(n_b):
        li = legend_styles[i][j]
        axi.plot([0.0+j, 0.5+j], [0.8*i, 0.8*i], color=li.get_color(), 
                 ls=li.get_linestyle(), alpha=li.get_alpha(), lw=li.get_linewidth())
for side in ["bottom", "left", "top", "right"]:
    axi.spines[side].set_visible(False)
axi.tick_params(axis="both", length=0, pad=3)
axi.set_xticks(np.arange(0.25, n_b+0.25, 2.0))
#axi.set_xticklabels([r"$\mathbf{\bar{m}}^i \cdot \mathbf{s}_a$", r"$\mathbf{\bar{m}} \cdot \mathbf{s}_b$"])
axi.set_xticklabels(list(range(0, n_b, 2)))
axi.set_xlabel(r"Odor $\gamma$", size=6, labelpad=3)
axi.xaxis.set_label_position('top') 
axi.set_yticks([0, 0.8, 1.6])
axi.invert_yaxis()
axi.set_yticklabels(["${}$".format(i) for i in i_highlights])
#axi.set_yticklabels(["Neuron 2", "Neuron 1"])
axi.set_ylabel("Neuron $i$", size=6, labelpad=1)

if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_lognormal_ibcm_m_series_narrower.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

## Panel C: dot products summary
To really show the specificity. 

In [ ]:
transient = int(3*tser_example.size/4)
hgammas_matrix = np.mean(hgammaser_example[transient:], axis=0)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*0.6, plt.rcParams["figure.figsize"][1])
# Extent: left, right, bottom, top
# Greyscale version
#ax.imshow(hgammas_matrix, cmap="Greys", aspect=0.6, extent=(0.5, n_b+0.5, 0.5, n_i_ibcm))
#ax.set_xticks(list(range(1, n_b+1)))
# Colorful version: add patches manually with fill_between. 
# Color highlighted neurons, leave others grayscale!
normed_matrix = (hgammas_matrix - hgammas_matrix.min()) / (hgammas_matrix.max() - hgammas_matrix.min())
for i in range(n_i_ibcm):
    # Full rainbow version
    #cmap = sns.light_palette(neuron_colors_full[i], as_cmap=True)
    # Version where only highlights are colored
    if i in i_highlights:
        cmap = sns.light_palette(neuron_colors3[i_highlights.index(i)], as_cmap=True)
    else:
        cmap = sns.color_palette("Greys", as_cmap=True)
    for j in range(n_b):
        ax.fill_between([-0.5+j, 0.5+j], -0.5+i, 0.5+i, color=cmap(normed_matrix[i, j]))
        
ax.set_xlim([-0.6, -0.6+n_b])
ax.set_ylim([-0.6, -0.6+n_i_ibcm])
ax.set_xticks(list(range(0, n_b)))
ax.set_yticks(list(range(0, n_i_ibcm, 2)))
for i, lbl in enumerate(ax.get_yticklabels()):
    if int(lbl.get_text()) in i_highlights:
        clr = neuron_colors3[i_highlights.index(int(lbl.get_text()))]
        lbl.set_color(clr)
        ax.yaxis.get_ticklines()[i].set_color(clr)
ax.set(xlabel=r"Component $\gamma$", ylabel="IBCM neuron index $i$")
cbar = fig.colorbar(mpl.cm.ScalarMappable(
    norm=mpl.colors.Normalize(hgammas_matrix.min(), hgammas_matrix.max()), 
    cmap="Greys"), ax=ax, label=r"Alignments $\bar{h}_{i\gamma}$ after 45 min", aspect=30, pad=0.1)
fig.tight_layout()
if do_save_plots:
    fig.savefig(os.path.join(panels_folder, "supfig_lognormal_ibcm_hgamma_matrix.pdf"), 
            transparent=True, bbox_inches="tight", bbox_extra_artists=(cbar.ax,))
plt.show()
plt.close()

## Panel D: $W$ series

In [ ]:
skp_lbl = n_orn // 4
#lbl = "i={}".format(i) if i % skp_lbl == 0 else ""
skp_plot = 20
tslice = slice(0, tser_example.shape[0], skp_plot)
t_axis = tser_example*dt_u/1000/60

fig, axes = plt.subplots(1, len(i_highlights), sharex=True, sharey=True)
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.5, plt.rcParams["figure.figsize"][1])
axes = axes.flatten()
# Make a palette for each highlighted neuron
neuron_comp_palettes = [sns.light_palette(neuron_colors3[i], n_colors=n_orn) for i in range(len(i_highlights))]
analytic_colors = [a[len(a)//2] for a in neuron_comp_palettes]
for i in range(n_orn):
    for j, ihigh in enumerate(i_highlights):
        ax = axes[j]
        li, = ax.plot(t_axis[tslice], wser_example[tslice, i, ihigh], zorder=analytical_w.size + i*n_orn + j, 
                      color=neuron_comp_palettes[j][i])
        lbl = "Analytical" if i == 0 else ""
        ax.axhline(analytical_w[i, ihigh], color=analytic_colors[j], ls="--", lw=0.65, label=lbl)
for j, ihigh in enumerate(i_highlights):
    axes[j].set_title("Neuron $j = {}$".format(ihigh))
    axes[j].set_xlabel("Time (min)")
axes[1].set_ylim(axes[1].get_ylim()[0], axes[1].get_ylim()[1]*1.15)
for j in range(len(i_highlights)):
    axes[j].legend(frameon=False, loc="upper center")
axes[0].set_ylabel(r"Inhibitory weights $W_{ij}$")
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_lognormal_ibcm_w_series.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

## Panels E-F: BioPCA learning of this background (L, alignment, off-diagonal)

In [ ]:
true_pca_values = ex["true_pca_vals"]
true_pca_vectors = ex["true_pca_vecs"]
learnt_pca_values = ex["learnt_pca_vals"]
learnt_pca_vectors = ex["learnt_pca_vecs"]
align_error_ser = ex["align_error_pca"]
yser_pca = ex["yser_pca"]
off_diag_l_pca = ex["off_diag_l_pca"]

In [ ]:
# First plot: eigenvalues
n_comp = learnt_pca_values.shape[1]
pca_palette = sns.color_palette("colorblind", n_colors=n_comp)
fig, ax = plt.subplots()

skp_plot = 4
tslice = slice(0, tser_example.shape[0], skp_plot)

for i in range(n_comp):
    li, = ax.plot(tser_example[tslice]*dt_u/1000/60, learnt_pca_values[tslice, i], label="Value {}".format(i),
                  lw=plt.rcParams["lines.linewidth"] - 0.5*i/n_comp, zorder=10-i, color=pca_palette[i])
    if true_pca_values[i] / true_pca_values.max() > 1e-12:
        ax.axhline(true_pca_values[i], ls="--", color=pca_palette[i], 
                   lw=plt.rcParams["lines.linewidth"] - 0.5*i/n_comp, zorder=n_comp-i)
ax.set(ylabel="Principal values (diag$(L^{-1})$)", yscale="log", xlabel="Time (min)")
handles = [mpl.lines.Line2D([0], [0], color="grey", ls="-", label=r"BioPCA ($L^{-1}$ diag.)", 
                            lw=plt.rcParams["lines.linewidth"]), 
          mpl.lines.Line2D([0], [0], color="grey", ls="--", label="True PCA", 
                          lw=plt.rcParams["lines.linewidth"])]
leg = ax.legend(handles=handles, frameon=False)
leg.set_zorder(30)
ax.set_ylim([ax.get_ylim()[0]*0.8, ax.get_ylim()[1]])

fig.tight_layout()
if do_save_plots:
    fig.savefig(os.path.join(panels_folder, "supfig_lognormal_biopca_eigenvalues.pdf"), 
            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
skp_plot = 4
tslice = slice(0, tser_example.shape[0], skp_plot)

fig, axes = plt.subplots(2, 1, sharex=True)
ax = axes[0]
ax.plot(tser_example[tslice]*dt_u/1000/60, align_error_ser[tslice], color="k")
ax.set(yscale="log", ylabel="Alignnment\nerror")

ax = axes[1]
ax.plot(tser_example[tslice]*dt_u/1000/60, np.mean(learnt_pca_values[tslice], axis=1), label="Diagonal", color=back_palette[0])
ax.plot(tser_example[tslice]*dt_u/1000/60, off_diag_l_pca[tslice], label="Off-diagonal", color=back_palette[1])
ax.legend(loc="upper right", bbox_to_anchor=(1.0, 1.1), frameon=False, handlelength=1.0, ncols=2, columnspacing=1.0)
ax.set(ylabel=r"$L_{ij}$ magnitude", xlabel="Time (min)", yscale="log")
fig.tight_layout()
if do_save_plots:
    fig.savefig(os.path.join(panels_folder, "supfig_lognormal_biopca_align_error.pdf"), 
            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

## Panel G: background inhibition of different models

In [ ]:
conc_ser_skp = conc_ser[::int(ex["skp"])]
bkvecser_example = conc_ser.dot(back_vecs_example)
yser_optimal = ex["yser_optimal"]
yser_avg = ex["yser_avg"]
yser_norms = {
    "ibcm": l2_norm(yser_ibcm, axis=1), 
    "biopca": l2_norm(yser_pca, axis=1), 
    "none": l2_norm(bkvecser_example, axis=1), 
    "avgsub": l2_norm(yser_avg, axis=1),
    "optimal": l2_norm(yser_optimal, axis=1)
}
model_order = ["none", "avgsub", "biopca", "ibcm", "optimal"]
skp_plot = 1
tslice = slice(0, tser_example.shape[0], skp_plot)

fig, ax = plt.subplots()
for m in model_order:
    ax.plot(t_axis[tslice], yser_norms[m][tslice], color=model_colors[m], label=model_nice_names[m], lw=0.5)
ax.set(xlabel="Time (min)", ylabel=r"PN activity norm, $\|\mathbf{y}\|$")
ax.legend(bbox_to_anchor=(0.5, 1.0), loc="upper center", frameon=False, title="Model", ncols=2, title_fontsize=7)
ylim = ax.get_ylim()
ax.set_ylim(ylim[0], ylim[1]+2.2)
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_lognormal_y_series.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

## Panels H-I: Jaccard with background vs with new odor

In [ ]:
ex_jac = np.load(pj(data_folder, "jaccards_onerun_lognormal_simulation.npz"))
jac_keys = list(ex_jac.keys())
jaccard_scores = {m: ex_jac[m] for m in jac_keys if not m.endswith("_back")}
jaccard_backs = {m: ex_jac[m] for m in jac_keys if m.endswith("_back")}
new_test_concs = ex["new_test_concs"]

In [ ]:
# Similarity with background odors vs with new odor: should be as low as possible
# Plot model histogram results
# One plot per new odor concentration
n_new_concs = len(new_test_concs)
fig, axes = plt.subplots(1, n_new_concs, sharex=True, sharey=True)
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*n_new_concs, plt.rcParams["figure.figsize"][1])
axes = axes.flatten()
models = ["none", "avgsub", "biopca", "ibcm", "optimal"]
# all_jacs indexed [new_odor, test_time, new_conc, back_sample
for m in models:  # Plot IBCM last
    # Take median across test times
    all_jacs = np.median(jaccard_scores[m], axis=[1, 3])
    all_back_jacs = np.median(jaccard_backs[m+"_back"], axis=[1, 3])
    for i in range(n_new_concs):
        axes[i].plot(all_back_jacs[:, i].flatten(), all_jacs[:, i].flatten(),
            marker="o", ls="none", ms=2.0, alpha=0.4, 
            label=model_nice_names.get(m, m), color=model_colors.get(m))

# Draw a diagonal on each graph
for i in range(n_new_concs):
    ylim = [0, 0.95]
    xlim = [0, 0.95]
    axes[i].plot(xlim, ylim, ls="--", color="grey", lw=1.0)
    axes[i].set_xlim(xlim)
    axes[i].set_ylim(ylim)

# Labeling the graphs, etc.
avg_conc = ex["moments_conc"][0]
for i in range(n_new_concs):
    ax = axes[i]
    axes[i].set_title("New conc. = {:.1f}".format(new_test_concs[i] / avg_conc) + r"$\langle c \rangle$", y=0.9)
    axes[i].set_xlabel("Jaccard with background\n(lower is better)")
    axes[i].set_ylabel("Jaccard with new odor\n(higher is better)")
axes[1].legend(loc="lower right", frameon=False)
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_lognormal_jaccards_background.pdf"),
                transparent=True, bbox_inches="tight")

plt.show()
plt.close()